In [ ]:
import pandas as pd
from osgeo import gdal
import geopandas as gpd
import zipfile
import os
import plotly.express as px
import json
import pyproj
import shapely.geometry
import h3
import plotly.graph_objects as go
import logging

# Configuring logging settings
logger = logging.getLogger(__name__)
logging.basicConfig(filename='EVEQ.log', filemode='w', level=logging.DEBUG)

# Suppress debug and info logs from external libraries including fiona and gdal
logging.getLogger('gdal').setLevel(logging.WARNING)

# Defining path to the .zip file
zip_path = 'EJSCREEN_2024_BG_with_AS_CNMI_GU_VI.gdb.zip'

# Extract the .zip file
extracted_path = 'EJSCREEN_2024_BG_with_AS_CNMI_GU_VI.gdb'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

# Joining path to the extracted .gdb directory
gdb_path = os.path.join(extracted_path, 'EJScreen_2024_BG_with_AS_CNMI_GU_VI.gdb')

# Loading the full data set layer into a GeoDataFrame
layer_name = 'EJSCREEN_Full_with_AS_CNMI_GU_VI' #this geo database has the EJSCREEN data layer and a US map layer
gdf = gpd.read_file(gdb_path, layer=layer_name)

# Display the first few rows of the GeoDataFrame
logger.info(f'First few rows of the GeoDataFrame:\n{gdf.head(2)}')

In [ ]:
# To create another dataframe to call back on if geometry errors occur
gdf1 = gdf

In [ ]:
#Do not run this cell unless the below cell returns "inf" values for geometry. Then run this cell and rerun conversion cell below
gdf = gdf1
logger.info(f'{gdf.head(2)}')

In [ ]:
#Converting to WGS84 to ensure proper coordinates
logger.info(f'{gdf.crs}')
gdf = gdf.to_crs(epsg=4326) 
logger.info('Ensure the gdf below has numeric lat/lon coordinates under the geometry column and does not just display "inf" repeatedly. If it displays inf, run the cell above and then rerun this cell')
logger.info(f'{gdf.head()}') 
logger.info(f'{gdf.crs}')
print(gdf.head(3))

In [ ]:
#Importing DOE/DOT identified DACs data

# Path to the DOE/DOT zip file
DOEDOT_zip_path = 'dacs_nevi_joint_May2022.zip'

# Extract the .zip file
DOEDOT_extracted_path = 'dacs_nevi_joint_May2022'
with zipfile.ZipFile(DOEDOT_zip_path, 'r') as DOEDOT_zip:
    DOEDOT_zip.extractall(DOEDOT_extracted_path)

# Path to the extracted directory
DOEDOT_path = os.path.join(DOEDOT_extracted_path, 'dacs_nevi_joint_May2022')

DOEDOT_gdf = gpd.read_file(DOEDOT_path)
logger.info(f'First few rows of the DOE/DOT GeoDataFrame:\n{DOEDOT_gdf.head(3)}')

In [ ]:
#Input the type of charging station. PureHDV if only HDV ports will be available. MixedPorts if MDHV and LDV ports available. Leave blank if either of these cases apply.
Station_Type = ("PureHDV")

In [ ]:
#Specify the number of block groups you want to display
block_groups = 20
logger.info(f'number of block groups {block_groups}')

In [ ]:
#Importing station location data into a geodataframe
OPT_STA_df = pd.read_csv('stations.csv') # stations.csv is an example of the accepted formatting
logger.info(f'Station Location Data:\n{OPT_STA_df.head(20)}')

In [ ]:
#Cleaning station data to just get where stations are to be built

#crete empty list to store built stations to
Stations = []

#Index through the optimal station dataframe, adding rows where built = 1 to station list
for index, row in OPT_STA_df.iterrows():
    if row["built"] == 1:
        Stations.append(row["hex"])
logger.info(f'{Stations}')

In [ ]:
# Create new relative disadvantage columns
gdf['Econ_DA'] = gdf[['P_LOWINCPCT', 'P_UNEMPPCT']].mean(axis=1)
gdf['Soc_DA'] = gdf[['P_PEOPCOLORPCT', 'P_DISABILITYPCT', 'P_LINGISOPCT', 'P_LESSHSPCT', 'P_OVER64PCT']].mean(axis=1)
gdf['Transp_DA'] = gdf[['P_PM25', 'P_OZONE', 'P_DSLPM', 'P_RSEI_AIR', 'P_PTRAF', 'P_NO2']].mean(axis=1)
gdf['Env_DA'] = gdf[['P_LIFEEXPPCT', 'P_LDPNT', 'P_PNPL', 'P_PRMP', 'P_PTSDF', 'P_UST', 'P_PWDIS']].mean(axis=1)
gdf['DAC_Ind'] = gdf[['Econ_DA', 'Soc_DA', 'Transp_DA', 'Env_DA']].sum(axis=1) / 4

# Display the first 10 rows of the updated GeoDataFrame
logger.info(f'Updated gdf with DAC Index info:\n{gdf.head()}')

#Note: Null values are not calculated in mean

In [ ]:
#Running a for loop over each of erics datapoints to find the nearest block groups to map

#Create an empty list to hold the subset from the gdf for each iteration
Sub = []
#Create empty lists to hold station latitude and longitude
Sta_Lat = []
Sta_Lon = []
    
# Re-project the GeoDataFrame to a suitable projected CRS (EPSG:3857) so the Distance function can be used. I think move this out of for loop
#print(gdf.crs)
gdf_projected1 = gdf.to_crs(epsg=3857)

for i in range(len(Stations)):
    # First testing ability to make and sort this new column for one point
    h = Stations[i]
    latlon = h3.h3_to_geo(h)
    
    # Separating latitude and longitude
    lat = latlon[0]
    lon = latlon[1]

    #Append Sta_Lat list adding lat for point and do same respectively for lon
    Sta_Lat.append(lat)
    Sta_Lon.append(lon)
    
    # Define a point with the lat/lon from the optimal charging station location data
    point = shapely.geometry.Point(lon, lat)  # Note: input longitude and then latitude
    
    # Re-project the point to the same projected CRS and extract that point
    point_projected = gpd.GeoSeries([point], crs="EPSG:4326").to_crs(epsg=3857).iloc[0]
    
    # Calculate the distance from the point to each geometry in the re-projected GeoDataFrame
    gdf_projected1['Distance'] = gdf_projected1.distance(point_projected)
    
    # Sort the GeoDataFrame by distance
    gdf_sorted = gdf_projected1.sort_values(by='Distance')
    
    # Create a subset with the desired number of block groups
    gdf_sub = gdf_sorted.iloc[0:block_groups].copy()

    #Append gdf_subset to sub list holding these gdf subsets
    Sub.append(gdf_sub)

#Concate the list back into a GeoDataFrame
gdf_subset = pd.concat(Sub, ignore_index=True)
logger.info(f'First few rows of gdf_subset:\n{gdf_subset.head(3)}')

In [ ]:
#Detects where data was not present for specific indicators and not included in the mean calculation for calculating the DAC_Index

for i in range(len(gdf_subset)):
    if pd.isnull(gdf_subset.loc[i, 'P_LOWINCPCT']):
        logger.info(f'Percentile for % low income not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')       
    if pd.isnull(gdf_subset.loc[i, 'P_UNEMPPCT']):
        logger.info(f'Percentile for % unemployed not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_PEOPCOLORPCT']):
        logger.info(f'Percentile for % people of color not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_DISABILITYPCT']):
        logger.info(f'Percentile for % persons with disabilities not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_LINGISOPCT']):
        logger.info(f'Percentile for % limited English speaking not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_LESSHSPCT']):
        logger.info(f'Percentile for % less than high school education not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_OVER64PCT']):
        logger.info(f'Percentile for % over age 64 not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_PM25']):
        logger.info(f'Percentile for Particulate Matter 2.5 not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_OZONE']):
        logger.info(f'Percentile for ozone not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_DSLPM']):
        logger.info(f'Percentile for diesel particulate matter not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_RSEI_AIR']):
        logger.info(f'Percentile for toxic releases to air not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_PTRAF']):
        logger.info(f'Percentile for traffic proximity not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_NO2']):
        logger.info(f'Percentile for nitrogen dioxide (NO2) not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_LIFEEXPPCT']):
        logger.info(f'Percentile for low life expectancy not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_LDPNT']):
        logger.info(f'Percentile for lead paint not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_PNPL']):
        logger.info(f'Percentile for superfund proximity not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_PRMP']):
        logger.info(f'Percentile for RMP facility proximity not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_PTSDF']):
        logger.info(f'Percentile for hazardous waste proximity not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_UST']):
        logger.info(f'Percentile for underground storage tanks not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')
    if pd.isnull(gdf_subset.loc[i, 'P_PWDIS']):
        logger.info(f'Percentile for wastewater discharge not included in calculated mean in block group: {gdf_subset.loc[i, "ID"]}, {gdf_subset.loc[i, "ST_ABBREV"]}, {gdf_subset.loc[i, "CNTY_NAME"]}')

In [ ]:
#This cell creates a plotly map for my DAC Index

# Calculate centroids to center map
centroids = gdf_subset.geometry.centroid

# Convert centroids back to WGS84 (EPSG:4326)
centroids = centroids.to_crs(epsg=4326)

# Calculate the center for the map
center_lat = centroids.y.mean()
center_lon = centroids.x.mean()
logger.info(f"Center Latitude: {center_lat}, Center Longitude: {center_lon}")

#Converting to WGS84 to ensure proper coordinates
gdf_subset = gdf_subset.to_crs(epsg=4326)  
    
# Convert the filtered GeoDataFrame to GeoJSON format
geojson = json.loads(gdf_subset.to_json())

# Create a DataFrame with the necessary columns
data = gdf_subset[['ID', 'DAC_Ind']].copy()

# Create the Plotly map
fig = px.choropleth_mapbox(
    data,
    geojson=geojson,
    locations='ID',
    featureidkey="properties.ID",  
    color='DAC_Ind',
    color_continuous_scale="Viridis",
    range_color=(data['DAC_Ind'].min(), data['DAC_Ind'].max()),
    mapbox_style="open-street-map",
    center={"lat": center_lat, "lon": center_lon},
    zoom=10,  # Adjust the zoom level if necessary
    opacity=0.5,
    labels={'DAC_Ind': 'DAC Index'}
)

#Comment or delete everything below for final
# Update the layout
fig.update_layout(
    title="DAC Index for Selected Block Groups",
    margin={"r": 0, "t": 0, "l": 0, "b": 0}
)

# Show the plot
#fig.show()

#Print nothing so jupyter notebook doesn't automatically generate a map since a fig.update was the last command
print()

In [ ]:
#Matching the selected block group subset with the appropriate DOE/DOT subsets off their census tracts

#Create an additional column on the gdf_subset which holds the First 9 digits of the block group ID
gdf_subset['First 9'] = gdf_subset['ID'].astype(str).str[:9]
logger.info(f'gdf_subset with first 9 digits of block group ID displayed in a column:\n{gdf_subset.head()}')

#Create an additional column in DOEDOT_gdf which holds the First 9 digits of the Census Tract GEOID
DOEDOT_gdf['FIRST 9'] = DOEDOT_gdf['GEOID'].astype(str).str[:9]
logger.info(f'DOEDOT_gdf with first 9 digits of census tract GEOID in column:\n{DOEDOT_gdf.head()}')

#Create empty list for DOEDOT subset
DOEDOT_sub = []

#Iterate over rows in DOEDOT_gdf to find where the first 9 digit of the ID and GEOID codes match and append those DOEDOT_gdf rows to the DOEDOT_sub list
for index, row in DOEDOT_gdf.iterrows():
    if row["FIRST 9"] in gdf_subset['First 9'].values:
        DOEDOT_sub.append(row)

#Convert list back to GeoDataFrame
DOEDOT_sub_gdf = gpd.GeoDataFrame(DOEDOT_sub, columns = DOEDOT_gdf.columns)

logger.info(f'DOEDOT_sub_gdf:\n{DOEDOT_sub_gdf.head()}')

In [ ]:
#Addressing null/invalid geometries
#Check for null geometries
logger.info(f'Number of null geometries: {DOEDOT_sub_gdf['geometry'].isnull().sum()}')

#Check for invalid geometries
logger.info(f'Number of invalid geometries: {(~DOEDOT_sub_gdf.is_valid).sum()}')

#Print invalid geometries
invalid_geom = DOEDOT_sub_gdf[(~DOEDOT_sub_gdf.is_valid)]
logger.info(f'GEOID(s) of invalid geometries: {invalid_geom['GEOID']}')

#Remove any any null geometries and print information about it
if DOEDOT_sub_gdf['geometry'].isnull().sum() > 0:
    null_geom = DOEDOT_sub_gdf[DOEDOT_sub_gdf['geometry'].isnull()]
    logger.info(f'GEOID(s) of null geometries: {null_geom['GEOID']}')
    logger.info(f'Number of rows before removing null geometries: {len(DOEDOT_sub_gdf)}')
    
    #Remove rows with null geometries
    DOEDOT_sub_gdf = DOEDOT_sub_gdf[DOEDOT_sub_gdf['geometry'].notnull()]
    logger.info(f'Number of rows after removing null geometries: {len(DOEDOT_sub_gdf)}')
    logger.info(f'Number of null geometries after removal: {DOEDOT_sub_gdf['geometry'].isnull().sum()}')
    logger.info(f'Number of invalid geometries after removing null geometries: {(~DOEDOT_sub_gdf.is_valid).sum()}')
else:
    pass

In [ ]:
#Adding DOE/DOT identified DAC layer to map

#Seperate DOE/DOT data into two data sets that are identified DACs and not identified DACs
NEVIDAC1 = DOEDOT_sub_gdf[DOEDOT_sub_gdf['NEVI_DAC'] == 1]
NEVIDAC0 = DOEDOT_sub_gdf[DOEDOT_sub_gdf['NEVI_DAC'] == 0]

#Convert to GeoJSON for graphing
DOEDOT_geojson1 = json.loads(NEVIDAC1.to_json())
DOEDOT_geojson0 = json.loads(NEVIDAC0.to_json())

# Create a DataFrame with the necessary columns
DOEDOT_data1 = NEVIDAC1[['GEOID', 'NEVI_DAC']].copy()
DOEDOT_data0 = NEVIDAC0[['GEOID', 'NEVI_DAC']].copy()

NEVI_DAC1_Layer = go.Choroplethmapbox(
    geojson=DOEDOT_geojson1,
    locations=NEVIDAC1['GEOID'],
    featureidkey="properties.GEOID",
    z=NEVIDAC1['NEVI_DAC'],
    colorscale=[(0, "rgba(255, 255, 255, 0)"), (1, "rgba(255, 255, 255, 0)")],  # Transparent color scale
    zmin=0,
    zmax=1,
    marker_opacity=1,  # Adjust to make it as transparent as possible
    marker_line_width=7,  # Default line width for NEVI_DAC == 0
    showscale=False  # Hide the color scale bar for this layer
)

NEVI_DAC0_Layer = go.Choroplethmapbox(
    geojson=DOEDOT_geojson0,
    locations=NEVIDAC0['GEOID'],
    featureidkey="properties.GEOID",
    z=NEVIDAC0['NEVI_DAC'],
    colorscale=[(0, "rgba(255, 255, 255, 0)"), (1, "rgba(255, 255, 255, 0)")],  # Transparent color scale
    zmin=0,
    zmax=1,
    marker_opacity=1,  # Adjust to make it as transparent as possible
    marker_line_width=2,  # Default line width for NEVI_DAC == 0
    showscale=False  # Hide the color scale bar for this layer
)

#Add these DOEDOT layers to the original figure
fig.add_trace(NEVI_DAC1_Layer)
fig.add_trace(NEVI_DAC0_Layer)

# Re-order the data so orignal layer with DAC Index on top and hovers
fig.data = (fig.data[2],fig.data[1],fig.data[0])

In [ ]:
# Create a layer that can graph all charging stations
Charging_Sta_Layer = go.Scattermapbox(
    lat=Sta_Lat,
    lon=Sta_Lon,
    mode='markers',
    marker=dict(
        size=5,
        color='red'
    ),
    name='Charging Stations'
)

#Add Charging Station Layer to figure
fig.add_trace(Charging_Sta_Layer)

#Updating map layout
fig.update_layout(
    title="Map of relative level of disadvantage around charging stations",
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    mapbox=dict(
        style="open-street-map",
        center={"lat": center_lat, "lon": center_lon},
        zoom=10
    )
)
fig.show()

In [ ]:
if Station_Type == "PureHDV":
    print("IMPORTANT: More research is needed, especially the incorporation of STAKEHOLDER FEEDBACK THROUGH COMMUNITY ENGAGEMENT.")
    print("The assumed best locations for charging stations with only HDV ports intended for long haul transportation HDVs are in areas with the lowest DAC Index scores. Chargers should be strategically placed so truck routes naturally route around communities as best as possible, especially communities with higher DAC Index scores.")
elif Station_Type == "MixedPorts":
    print("IMPORTANT: More research is needed, especially the incorporation of STAKEHOLDER FEEDBACK THROUGH COMMUNITY ENGAGEMENT.")
    print("The assumed best locations are on the fringe of areas with higher DAC Index scores. Chargers should be strategically placed so truck routes naturally route around communities while improving accessibility to LDV charging as best as possible, especially for communities with higher DAC Index scores.")
else:
    print("Collect stakeholder feedback through community engagement to find the best locations to build desired EVSE.")

In [ ]:
#End of code, below only need to be run if more information on where missing data for the entire datasets is desired

In [ ]:
# Print the number of null values for each indicator used in the mean calculations. This does not need to be run and can be deleted if desired. Only for informative purposes

# Economic Data (Econ_DA)
print("Number of null P_LOWINCPCT:", gdf['P_LOWINCPCT'].isnull().sum())
print("Number of null P_UNEMPPCT:", gdf['P_UNEMPPCT'].isnull().sum())

# Social Data (Soc_DA)
print("Number of null P_PEOPCOLORPCT:", gdf['P_PEOPCOLORPCT'].isnull().sum())
print("Number of null P_DISABILITYPCT:", gdf['P_DISABILITYPCT'].isnull().sum())
print("Number of null P_LINGISOPCT:", gdf['P_LINGISOPCT'].isnull().sum())
print("Number of null P_LESSHSPCT:", gdf['P_LESSHSPCT'].isnull().sum())
print("Number of null P_OVER64PCT:", gdf['P_OVER64PCT'].isnull().sum())

# Transportation Data (Transp_DA)
print("Number of null P_PM25:", gdf['P_PM25'].isnull().sum())
print("Number of null P_OZONE:", gdf['P_OZONE'].isnull().sum())
print("Number of null P_DSLPM:", gdf['P_DSLPM'].isnull().sum())
print("Number of null P_RSEI_AIR:", gdf['P_RSEI_AIR'].isnull().sum())
print("Number of null P_PTRAF:", gdf['P_PTRAF'].isnull().sum())
print("Number of null P_NO2:", gdf['P_NO2'].isnull().sum())

# Environmental Data (Env_DA)
print("Number of null P_LIFEEXPPCT:", gdf['P_LIFEEXPPCT'].isnull().sum())
print("Number of null P_LDPNT:", gdf['P_LDPNT'].isnull().sum())
print("Number of null P_PNPL:", gdf['P_PNPL'].isnull().sum())
print("Number of null P_PRMP:", gdf['P_PRMP'].isnull().sum())
print("Number of null P_PTSDF:", gdf['P_PTSDF'].isnull().sum())
print("Number of null P_UST:", gdf['P_UST'].isnull().sum())
print("Number of null P_PWDIS:", gdf['P_PWDIS'].isnull().sum())


In [ ]:
#viewing null/invalid geometries for entire DOEDOT gdf. Not necessarry to run
#Check for null geometries
print("Number of null geometries:", DOEDOT_gdf['geometry'].isnull().sum())

#Check for invalid geometries
print("Number of invalid geometries:", (~DOEDOT_gdf.is_valid).sum())

print(len(DOEDOT_gdf))